In [61]:
import sys
import os
import re

path = "ConvertToBin"
isFile = os.path.isfile(path)
fileList = []
outFileNames_vm = []
tokenIndex = 1
ind = "  "
classSymbolTable = {}
classSymbolCounts = {'static':0,'field':0}

subroutineSymbolCounts = {'local':0,'argument':0}
subroutineSymbolTable = {}
subroutineType = ""
classname = ""
functionname = ""

code = ""
expressionListCounter = 0
paramListCounter = 0

dict_keywords = {'class':'keyword','constructor':'keyword','function':'keyword',
        'method':'keyword','field':'keyword','static':'keyword','var':'keyword','int':'keyword',
        'char':'keyword','boolean':'keyword','void':'keyword','true':'keyword','false':'keyword','null':'keyword',
        'this':'keyword','let':'keyword','do':'keyword','if':'keyword','else':'keyword',
        'while':'keyword','return':'keyword'}
        
symbol_keywords ={'{':'symbol','}':'symbol','(':'symbol',')':'symbol','[':'symbol',
        ']':'symbol','.':'symbol',',':'symbol',';':'symbol','+':'symbol',
        '-':'symbol','*':'symbol','/':'symbol','&':'symbol','|':'symbol',
        '<':'symbol','>':'symbol','=':'symbol','~':'symbol'}

specialCases = {'<':'&lt;','>':'&gt;','&':'&amp;'}

keyword = '(class|constructor|function|method|static|field|var|int|char|boolean|void|true|false|null|this|let|do|if|else|while|return)'
symbol = '[{}()[\].,;+\-*/&|<>=~]' #forgot escape for ]
integerConstant = '\d+'
stringConstant = '"[^"\n]*"'
identifier = '[\w]+'

#files to cover
if isFile:
    fileList.append(path)
    outFileNames_vm.append(path[0:(len(path)-5)] + '.vm')
else:
    for filename in os.listdir(path):
        if filename.endswith(".jack"):
            fileList.append(path+"/"+filename)
            outFileNames_vm.append(path + "/" + filename[0:len(filename)-5] + '.vm')
        else:
            continue

def removeComments(line):
    line = line.strip()
    if line == '' or line[0:2] == '//' or line[0:2] == "/*" or line[0] == "*":
        return ""
    result = line.split("//")
    return result[0]

def tokenizer(textToTokenize):
    regex = re.compile(symbol+"|"+identifier+"|"+stringConstant+"|"+integerConstant)
    elems = regex.findall(textToTokenize)
    return elems   

def makeTxml(tokens):
    txml = "<tokens>\n"
    for token in tokens:
        newLine = ""
        if token in dict_keywords:
            newLine = "<keyword> "+token+" </keyword>\n"
        elif token in symbol_keywords:
            if token in specialCases:
                newLine = "<symbol> "+specialCases.get(token)+" </symbol>\n"
            else:
                newLine = "<symbol> "+token+" </symbol>\n"
        elif re.match(integerConstant,token):
            newLine = "<integerConstant> "+token+" </integerConstant>\n"
        elif re.match(stringConstant,token):
            newLine = "<stringConstant> "+token.replace('"','')+" </stringConstant>\n"
        elif re.match(identifier,token):
            newLine = "<identifier> "+token+" </identifier>\n"
        txml = txml + newLine
    txml = txml + "</tokens>"
    return txml

def compilationEngine(tokens, txml, indent):
    global tokenIndex
    xml = "<class>\n"
    xml = xml + compileClass(tokens, txml, indent+1)
    xml = xml + "</class>\n"
    return xml

def updateClassSymbolTable(name, typeof, kind):
    global classSymbolTable
    global classSymbolCounts
    classSymbolTable[name] = (typeof, kind, classSymbolCounts[kind])
    classSymbolCounts[kind] = classSymbolCounts[kind] + 1
    
def compileClass(tokens, txml, indent):
    global tokenIndex
    result = indent*ind + txml[tokenIndex] + "\n" #class
    result += indent*ind + txml[tokenIndex+1] + "\n" #main
    result += indent*ind + txml[tokenIndex+2] + "\n" #{
    tokenIndex = tokenIndex+3
    while tokens[tokenIndex-1] == "field" or tokens[tokenIndex-1] == "static" :
        typeof = tokens[tokenIndex]
        kind = tokens[tokenIndex-1]
        updateClassSymbolTable(tokens[tokenIndex+1], typeof, kind)
        result += indent*ind + "<classVarDec>" + "\n"
        result += compileClassVarDec(txml, tokens, indent, typeof, kind)
        result += indent*ind + "</classVarDec>" + "\n"
    while (tokens[tokenIndex-1] == "constructor" or tokens[tokenIndex-1] == "function" or tokens[tokenIndex-1] == "method"):
        result += indent*ind + "<subroutineDec>" + "\n"
        result += compileSubroutineDec(txml, tokens, indent);
        result += indent*ind + "</subroutineDec>" + "\n"
    result += indent*ind + txml[tokenIndex] + "\n" #}
    return result

def compileClassVarDec(txml, tokens, indent, typeof, kind):
    global tokenIndex
    result = ""
    indent = indent + 1
    while(tokens[tokenIndex-1] != ";"):
        if(tokens[tokenIndex] == ","):
            updateClassSymbolTable(tokens[tokenIndex+1], typeof, kind)
        result += indent*ind + txml[tokenIndex] + "\n"
        tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #appends ;
    tokenIndex = tokenIndex + 1
    return result

def compileSubroutineDec(txml, tokens, indent):
    global tokenIndex
    global subroutineSymbolTable
    global subroutineSymbolCounts
    global subroutineType
    global code
    global functionname
    result = ""
    indent = indent + 1
    #
    subroutineType = tokens[tokenIndex-1]
    subroutineSymbolCounts = {'local':0,'argument':0}
    subroutineSymbolTable = {}
    if(subroutineType == "method"):
        subroutineSymbolTable['this'] = (classname, 'argument', subroutineSymbolCounts['argument'])
        subroutineSymbolCounts['argument'] = subroutineSymbolCounts['argument'] + 1       
    #
    result += indent*ind + txml[tokenIndex] + "\n" #constructor/function/method
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #return type
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #parameter name
    # <vm>
    functionname = classname + "." + tokens[tokenIndex-1]
    #</vm>
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #(
    tokenIndex = tokenIndex + 1
    result += indent*ind +"<parameterList>"+"\n"
    result += compileParameterList(txml, tokens, indent)
    result += indent*ind +"</parameterList>"+"\n"
    result += indent*ind + txml[tokenIndex] + "\n" #)
    tokenIndex = tokenIndex + 1
    result += indent*ind +"<subroutineBody>"+"\n"
    result += compileSubroutineBody(txml, tokens, indent)
    result += indent*ind +"</subroutineBody>"+"\n"
    #print(subroutineSymbolTable)
    return result;

def compileParameterList(txml, tokens, indent):
    global tokenIndex
    global subroutineSymbolTable
    global subroutineSymbolCounts
    global subroutineType
    global paramListCounter
    indent = indent + 1
    result = ""
    paramListCounter = 0
    while(tokens[tokenIndex-1] != ")"):
        if("identifier" in txml[tokenIndex]):
            subroutineSymbolTable[tokens[tokenIndex-1]] = (tokens[tokenIndex-2], 'argument', subroutineSymbolCounts['argument'])
            subroutineSymbolCounts['argument'] = subroutineSymbolCounts['argument'] + 1
        result += indent*ind + txml[tokenIndex] + "\n"
        tokenIndex = tokenIndex + 1
        paramListCounter += 1
    return result

def compileSubroutineBody(txml, tokens, indent):
    global tokenIndex
    global subroutineSymbolTable
    global subroutineSymbolCounts
    global subroutineType
    global functionname
    global code
    indent = indent + 1
    result = indent*ind + txml[tokenIndex] + "\n" #{
    tokenIndex = tokenIndex + 1
    opencnt = 1
    while tokens[tokenIndex-1] == "var":
        result += indent*ind + "<varDec>" + "\n"
        result += compileVarDec(txml, tokens, indent)
        result += indent*ind + "</varDec>" + "\n"
    code += function(functionname, subroutineSymbolCounts['local'])
    result += indent*ind + "<statements>" + "\n"
    result += compileStatements(txml, tokens, indent)
    result += indent*ind + "</statements>"+ "\n"   
    result += indent*ind + txml[tokenIndex] + "\n" #}
    tokenIndex = tokenIndex+ 1
    return result

def compileVarDec(txml, tokens, indent):
    global tokenIndex
    global subroutineSymbolTable
    global subroutineSymbolCounts
    global subroutineType
    result = ""
    indent = indent + 1
    result += indent*ind + txml[tokenIndex] + "\n" #var
    tokenIndex = tokenIndex + 1
    typeof = ""
    while(tokens[tokenIndex-1] != ";"):
        if "identifier" in txml[tokenIndex]:
            if(typeof == ""):
                typeof = tokens[tokenIndex-2]
                subroutineSymbolTable[tokens[tokenIndex-1]] = (typeof, 'local', subroutineSymbolCounts['local'])
                subroutineSymbolCounts['local'] = subroutineSymbolCounts['local'] + 1
            else:    
                subroutineSymbolTable[tokens[tokenIndex-1]] = (typeof, 'local', subroutineSymbolCounts['local'])
                subroutineSymbolCounts['local'] = subroutineSymbolCounts['local'] + 1
        #
        result += indent*ind + txml[tokenIndex] + "\n"
        tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #appends ;
    tokenIndex = tokenIndex + 1
    return result

def compileStatements(txml, tokens, indent):
    global tokenIndex
    result = ""
    indent = indent + 1
    while tokens[tokenIndex-1] == "let" or tokens[tokenIndex-1] == "if" or tokens[tokenIndex-1] == "while" or tokens[tokenIndex-1] == "do" or tokens[tokenIndex-1] == "return":
        if tokens[tokenIndex-1] == "let":
            result += indent*ind + "<letStatement>"+"\n"
            result += compileLet(txml, tokens, indent)
            result += indent*ind + "</letStatement>"+"\n"
        elif tokens[tokenIndex-1] == "if":
            result += indent*ind + "<ifStatement>"+"\n"
            result += compileIf(txml, tokens, indent)
            result += indent*ind + "</ifStatement>"+"\n"
        elif tokens[tokenIndex-1] == "while":
            result += indent*ind + "<whileStatement>"+"\n"
            result += compileWhile(txml, tokens, indent)
            result += indent*ind + "</whileStatement>"+"\n"
        elif tokens[tokenIndex-1] == "do":
            result += indent*ind + "<doStatement>"+"\n"
            result += compileDo(txml, tokens, indent)
            result += indent*ind + "</doStatement>"+"\n"
        elif tokens[tokenIndex-1] == "return":
            result += indent*ind + "<returnStatement>"+"\n"
            result += compileReturn(txml, tokens, indent)
            result += indent*ind + "</returnStatement>"+"\n"
    return result

def compileLet(txml, tokens, indent):
    global tokenIndex
    global code
    global classSymbolTable
    global classSymbolCounts
    global subroutineSymbolTable
    global subroutineSymbolCounts
    indent = indent + 1
    result = indent*ind + txml[tokenIndex] + "\n" #let
    tokenIndex = tokenIndex + 1
    var = tokens[tokenIndex-1]
    result += indent*ind + txml[tokenIndex] + "\n" #var
    tokenIndex = tokenIndex + 1
    if tokens[tokenIndex-1] == "[":
        result += indent*ind + txml[tokenIndex] + "\n" #[
        tokenIndex = tokenIndex + 1
        result += indent*ind + "<expression>" + "\n" 
        result += compileExpression(txml, tokens, indent)
        result += indent*ind + "</expression>" + "\n" 
        result += indent*ind + txml[tokenIndex] + "\n" #]
        tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #=
    tokenIndex = tokenIndex + 1
    result += indent*ind + "<expression>" + "\n"
    result += compileExpression(txml, tokens, indent)
    result += indent*ind + "</expression>" + "\n"
    result += indent*ind + txml[tokenIndex] + "\n" #;
    tokenIndex = tokenIndex + 1
    #first check if var is class variable. if not - it should be method var
    #for keys,values in classSymbolTable.items():
    #    print(keys)
    if var in classSymbolTable:
        pass
        #print(var)
        code += pop(classSymbolTable[var][1], classSymbolCounts[classSymbolTable[var][1]])
    else:
        #print(var)
        #print(subroutineSymbolTable[var][1])
        #print(subroutineSymbolCounts)
        #print(subroutineSymbolCounts[subroutineSymbolTable[var][1]])
        code += pop(subroutineSymbolTable[var][1], subroutineSymbolCounts[subroutineSymbolTable[var][1]])
    return result

def compileIf(txml, tokens, indent):
    global tokenIndex
    indent = indent + 1 
    result = indent*ind + txml[tokenIndex] + "\n" #if
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #(
    tokenIndex = tokenIndex + 1
    result += indent*ind + "<expression>" + "\n"
    result += compileExpression(txml, tokens, indent)
    result += indent*ind + "</expression>" + "\n"
    result += indent*ind + txml[tokenIndex] + "\n" #)
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #{
    tokenIndex = tokenIndex + 1
    #
    result += indent*ind + "<statements>" + "\n"
    result += compileStatements(txml, tokens, indent)
    result += indent*ind + "</statements>" + "\n"
    #
    result += indent*ind + txml[tokenIndex] + "\n" #}
    tokenIndex = tokenIndex + 1
    if tokens[tokenIndex-1] == "else":
        result += indent*ind + txml[tokenIndex] + "\n" #else
        tokenIndex = tokenIndex + 1
        result += indent*ind + txml[tokenIndex] + "\n" #{
        tokenIndex = tokenIndex + 1
        #
        result += indent*ind + "<statements>" + "\n"
        result += compileStatements(txml, tokens, indent)
        result += indent*ind + "</statements>" + "\n"
        #
        result += indent*ind + txml[tokenIndex] + "\n" #}
        tokenIndex = tokenIndex + 1
    #tokenIndex = tokenIndex + 1
    return result

def compileWhile(txml, tokens, indent):
    global tokenIndex
    result = ""
    indent = indent + 1
    result = indent*ind + txml[tokenIndex] + "\n" #while
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #(
    tokenIndex = tokenIndex + 1
    result += indent*ind + "<expression>" + "\n"
    result += compileExpression(txml, tokens, indent)
    result += indent*ind + "</expression>" + "\n"
    result += indent*ind + txml[tokenIndex] + "\n" #)
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #{
    tokenIndex = tokenIndex + 1
    #
    result += indent*ind + "<statements>" + "\n"
    result += compileStatements(txml, tokens, indent)
    result += indent*ind + "</statements>" + "\n"
    #
    result += indent*ind + txml[tokenIndex] + "\n" #}
    tokenIndex = tokenIndex + 1
    return result

def compileDo(txml, tokens, indent):
    global tokenIndex
    global code
    global expressionListCounter
    indent = indent + 1
    result = indent*ind + txml[tokenIndex] + "\n" #do
    tokenIndex = tokenIndex + 1
    functionname = tokens[tokenIndex-1]
    result += indent*ind + txml[tokenIndex] + "\n" #var/routine
    tokenIndex = tokenIndex + 1
    if tokens[tokenIndex-1] == ".":
        result += indent*ind + txml[tokenIndex] + "\n" #.
        tokenIndex = tokenIndex + 1
        functionname += "." + tokens[tokenIndex-1]
        result += indent*ind + txml[tokenIndex] + "\n" #subroutine
        tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #(
    tokenIndex = tokenIndex + 1
    #
    result += indent*ind + "<expressionList>" + "\n"
    result += compileExpressionList(txml, tokens, indent)
    result += indent*ind + "</expressionList>" + "\n"
    #
    result += indent*ind + txml[tokenIndex] + "\n" #)
    tokenIndex = tokenIndex + 1
    result += indent*ind + txml[tokenIndex] + "\n" #;
    tokenIndex = tokenIndex + 1
    code += call(functionname, expressionListCounter)
    code += pop('temp', 0)
    expressionListCounter = 0
    return result

def compileReturn(txml, tokens, indent):
    global tokenIndex
    global code
    indent = indent + 1
    result = indent*ind + txml[tokenIndex] + "\n" #return
    tokenIndex = tokenIndex + 1
    if tokens[tokenIndex-1] != ";":
        result += indent*ind + "<expression>" + "\n"
        result += compileExpression(txml, tokens, indent)
        result += indent*ind + "</expression>" + "\n"
    else:
        code += push('constant',0)
    result += indent*ind + txml[tokenIndex] + "\n" #;
    tokenIndex = tokenIndex + 1
    code += ret()
    return result

def compileExpression(txml, tokens, indent):
    global tokenIndex
    global code
    indent = indent + 1
    result = indent*ind + "<term>" + "\n"
    result += compileTerm(txml, tokens, indent)
    result += indent*ind + "</term>" + "\n"
    while tokens[tokenIndex-1] in ['+','-','*','/','&','|','<','>','=']:
        op = tokens[tokenIndex-1]
        result += indent*ind + txml[tokenIndex] + "\n" #op 
        tokenIndex = tokenIndex + 1
        result += indent*ind + "<term>" + "\n"
        result += compileTerm(txml, tokens, indent)
        result += indent*ind + "</term>" + "\n"
        code += calc(op)
    return result

def compileTerm(txml, tokens, indent):
    global tokenIndex
    global code
    indent = indent + 1
    result = ""
    if tokens[tokenIndex-1] == "(": #meaning its expression
        result += indent*ind + txml[tokenIndex] + "\n" #(
        tokenIndex = tokenIndex + 1
        result += indent*ind + "<expression>" + "\n"
        result += compileExpression(txml, tokens, indent)
        result += indent*ind + "</expression>" + "\n"
        result += indent*ind + txml[tokenIndex] + "\n" #)
        tokenIndex = tokenIndex + 1
    elif tokens[tokenIndex-1] in ['-','~']:
        symbol = tokens[tokenIndex-1]
        result += indent*ind + txml[tokenIndex] + "\n" #- or ~
        tokenIndex = tokenIndex + 1
        result += indent*ind + "<term>" + "\n"
        result += compileTerm(txml, tokens, indent)
        result += indent*ind + "</term>" + "\n"
        #vm
        if(symbol == '-'):
            code += "neg\n"
        else:
            code += "not\n"
        #/vm
    else:
        if("Constant" in txml[tokenIndex]):
            code += push("constant", tokens[tokenIndex-1])
        result += indent*ind + txml[tokenIndex] + "\n" #any other thing: constants, expressionList, subroutineCall
        tokenIndex = tokenIndex + 1
        if tokens[tokenIndex-1] == "(":
            result += indent*ind + txml[tokenIndex] + "\n" #(
            tokenIndex = tokenIndex + 1
            result += indent*ind + "<expressionList>" + "\n"
            result += compileExpressionList(txml, tokens, indent)
            result += indent*ind + "</expressionList>" + "\n"
            result += indent*ind + txml[tokenIndex] + "\n" #)
            tokenIndex = tokenIndex + 1
        elif tokens[tokenIndex-1] == "[": #expression
            result += indent*ind + txml[tokenIndex] + "\n" #[
            tokenIndex = tokenIndex + 1
            result += indent*ind + "<expression>" + "\n"
            result += compileExpression(txml, tokens, indent)
            result += indent*ind + "</expression>" + "\n"
            result += indent*ind + txml[tokenIndex] + "\n" #]
            tokenIndex = tokenIndex + 1
        elif tokens[tokenIndex-1] == ".": #subroutine
            toAdd = tokens[tokenIndex-2]+tokens[tokenIndex-1]+tokens[tokenIndex]  
            result += indent*ind + txml[tokenIndex] + "\n" #should be .
            tokenIndex = tokenIndex + 1
            result += indent*ind + txml[tokenIndex] + "\n" #name
            tokenIndex = tokenIndex + 1
            result += indent*ind + txml[tokenIndex] + "\n" #[ arada (
            tokenIndex = tokenIndex + 1
            result += indent*ind + "<expressionList>" + "\n"
            result += compileExpressionList(txml, tokens, indent)
            result += indent*ind + "</expressionList>" + "\n"
            result += indent*ind + txml[tokenIndex] + "\n" #] arada )
            tokenIndex = tokenIndex + 1
            code += call(toAdd, expressionListCounter)
    return result

def compileExpressionList(txml, tokens, indent):
    global tokenIndex
    global expressionListCounter
    result = ""
    indent = indent + 1
    if tokens[tokenIndex-1] != ")": #eseigi expression
        result += indent*ind + "<expression>" + "\n"
        result += compileExpression(txml, tokens, indent)
        result += indent*ind + "</expression>" + "\n"
        expressionListCounter += 1
    while tokens[tokenIndex-1] != ")":
        result += indent*ind + txml[tokenIndex] + "\n" #,
        tokenIndex = tokenIndex + 1
        result += indent*ind + "<expression>" + "\n"
        result += compileExpression(txml, tokens, indent)
        result += indent*ind + "</expression>" + "\n"
        expressionListCounter += 1
    return result



def push(typeof, val):
    result = "push " + typeof + " " + str(val) + "\n"
    return result

def pop(typeof, val):
    result =  "pop " + typeof + " " + str(val) + "\n"
    return result

def ret():
    return 'return\n'

def function(name, var):
    return "function " + name + " " + str(var) + "\n"

def calc(op):
    operator_map = {'+':'add','-':'sub', '*':'call Math.multiply 2',
                   '/':'call Math.divide 2', '&':'and', '|':'or', '<':'lt',
                   '>':'gt', '=':'eq'}
    return operator_map[op] + "\n"

def label(name):
    return "label " + name + "\n"

def goto(label):
    return "goto " + label + "\n"

def ifGoto(label):
    return "if-goto " + label + "\n"

def call(func, arg):
    return "call " + func + " " + str(arg) + "\n"




for file,outfile in zip(fileList,outFileNames_vm):
    fileIn = open(file,'r') 
    fileOut = open(outfile,'w')
    rows = fileIn.readlines()
    textToTokenize = ""
    for line in rows:
        textToTokenize = textToTokenize + removeComments(line)
    textToTokenize = ' '.join(textToTokenize.split())
    tokens = tokenizer(textToTokenize)
    txml = makeTxml(tokens)
    tokenIndex = 1
    #
    classSymbolTable = {}
    classSymbolCounts = {'static':0,'field':0,'var':0,'argument':0}
    classname = file.split("/")
    classname = classname[len(classname)-1]
    classname = classname[0:len(classname)-5]
    code = ""
    expressionListCounter = 0
    #
    #print(file)
    xml = compilationEngine(tokens, txml.splitlines(), 0)
    print(file)
    print(code)
    #print(classSymbolTable)
    fileOut.write(code)
    fileIn.close()
    fileOut.close()

value
local
{'local': 1, 'argument': 0}
loop
local
{'local': 3, 'argument': 1}
position
local
{'local': 3, 'argument': 1}
mask
local
{'local': 3, 'argument': 1}
loop
local
{'local': 3, 'argument': 1}
length
argument
{'local': 0, 'argument': 3}
startAddress
argument
{'local': 0, 'argument': 3}
ConvertToBin/Main.jack
function Main.main 1
push constant 8001
push constant 16
push constant 1
neg
call Main.fillMemory 3
pop temp 0
push constant 8000
call Memory.peek 1
call Main.convert 2
pop temp 0
push constant 0
return
function Main.convert 3
push constant 1
add
call Main.nextMask 1
push constant 16
gt
not
and
push constant 0
eq
not
push constant 8000
add
push constant 1
call Memory.poke 3
pop temp 0
push constant 8000
add
push constant 0
call Memory.poke 2
pop temp 0
push constant 0
return
function Main.nextMask 0
push constant 0
eq
push constant 1
return
push constant 2
call Math.multiply 2
return
function Main.fillMemory 0
push constant 0
gt
call Memory.poke 2
pop temp 0
push constant 1
